In [9]:
import pydub
import numpy as np
import os
import sounddevice as sd

In [7]:
audios=[]
def find_(name:str):
    result=[]
    for idx,ch in enumerate(name):
        if ch=="_":
            result.append(idx)
    return result
for dirpath, dirnames, filenames in os.walk("./audio1"):
    for file_name in filenames:
        if file_name==".DS_Store":
            continue
        audio_data={}
        audio_data["format"]=file_name[-3:]
        i_=find_(file_name)
        audio_data["type"]=file_name[:i_[0]]
        # print(audio_data["type"])
        # print(audio_data["freq"])
        audio_data["bits"]=file_name[i_[-1]+1:-4]
        # print(audio_data["bits"])
        audios.append(audio_data)
        full_path=os.path.join(dirpath, file_name)
        dub_temp=pydub.AudioSegment.from_file(full_path, format=audio_data["format"]) 
        samples=dub_temp.get_array_of_samples()
        sample_rate=dub_temp.frame_rate
        audio_data["freq"]=sample_rate
        audio_data["data"]=np.array(samples)
        audios.append(audio_data)

In [10]:
sd.play(audios[0]["data"], audios[0]["freq"])

ModuleNotFoundError: No module named 'PySoundFile'